# Introduction

This project is adapted from an R-based analysis provided by DQLab Finance case “Proses Investasi Investor” and reimplemented in Python.

This analysis examines how investors progress from registration to repeated investments on a peer-to-peer lending platform.

The dataset is provided by DQLab (Bahasa Indonesia) and originally implemented in R.
This notebook presents a Python-based analysis focusing on investor conversion and retention behavior. 

## Data Overview

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

loans = pd.read_csv("event.csv")
loans["created_at"] = pd.to_datetime(loans["created_at"])

## Investment Funnel Analysis

This section summarizes how many unique investors and loans appear at each key event in the investment process.

The results show a significant drop between loan marketplace exposure and completed investments, indicating friction in the conversion funnel.


In [9]:
event_summary = (
    loans
    .groupby("nama_event")
    .agg(
        unique_loans=("loan_id", "nunique"),
        unique_investors=("investor_id", "nunique")
    )
    .reset_index()
)

event_summary

,nama_event,unique_loans,unique_investors
0,investor_order_loan,3641,804
1,investor_pay_loan,3632,771
2,investor_register,0,17931
3,investor_view_loan,3678,1095
4,loan_to_marketplace,3678,0


- investor_view_loan — shows interest
- investor_order_loan — orders placed
- investor_pay_loan — actual investments
- loan_to_marketplace — loan availability only

A loan is uploaded to the marketplace → investors view loan details → investors make an order → investors pay (investment completed).
Understanding the conversion at each step helps identify friction points in the funnel.

## Time to First Investment

This analysis measures how long registered investors take to make their first investment.

The distribution shows that most converted investors make their first investment within the first few months, while a large portion never convert at all.

In [4]:
register = loans[loans["nama_event"] == "investor_register"][
    ["investor_id", "created_at"]
]

first_invest = (
    loans[loans["nama_event"] == "investor_pay_loan"]
    .groupby("investor_id")["created_at"]
    .min()
    .reset_index(name="first_invest_date")
)

conversion = register.merge(first_invest, on="investor_id", how="left")

conversion["months_to_invest"] = (
    (conversion["first_invest_date"] - conversion["created_at"])
    .dt.days // 30
)

## Cohort Retention Analysis

Investors are grouped into cohorts based on the month of their first investment.
Retention is measured as the percentage of investors who continue investing in subsequent months.

The cohort table shows a sharp decline after the first investment month, suggesting that repeat investment behavior is limited and early engagement is critical.

In [5]:
invest_events = loans[loans["nama_event"] == "investor_pay_loan"][
    ["investor_id", "created_at"]
]

invest_events = invest_events.merge(
    first_invest,
    on="investor_id",
    how="inner"
)

invest_events["months_since_first"] = (
    (invest_events["created_at"] - invest_events["first_invest_date"])
    .dt.days // 30
)

invest_events["cohort"] = invest_events["first_invest_date"].dt.to_period("M")

In [6]:
cohort_data = (
    invest_events
    .groupby(["cohort", "months_since_first"])
    ["investor_id"]
    .nunique()
    .reset_index(name="investor_count")
)

In [7]:
cohort_data["cohort_size"] = (
    cohort_data
    .groupby("cohort")["investor_count"]
    .transform("max")
)

cohort_data["retention_rate"] = (
    cohort_data["investor_count"] / cohort_data["cohort_size"]
)

retention_table = (
    cohort_data
    .pivot(
        index="cohort",
        columns="months_since_first",
        values="retention_rate"
    )
    .drop(columns=0)
)

retention_table

months_since_first,1,2,3,4,5,6,7,8,10
cohort,,,,,,,,,
2019-07,0.258065,0.258065,0.193548,0.064516,0.161290,0.161290,0.193548,0.129032,0.032258
2019-08,0.352941,0.196078,0.254902,0.196078,0.196078,0.215686,0.098039,0.019608,NaN
2019-09,0.257143,0.185714,0.185714,0.157143,0.185714,0.100000,0.014286,NaN,NaN
2019-10,0.325000,0.287500,0.175000,0.237500,0.087500,0.062500,NaN,NaN,NaN
2019-11,0.303030,0.242424,0.242424,0.080808,0.070707,0.010101,NaN,NaN,NaN
2019-12,0.380952,0.301587,0.031746,0.047619,0.015873,NaN,NaN,NaN,NaN
2020-01,0.323944,0.126761,0.042254,0.014085,NaN,NaN,NaN,NaN,NaN
2020-02,0.165217,0.034783,0.008696,NaN,NaN,NaN,NaN,NaN,NaN
2020-03,0.107843,0.009804,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Key Insights

- Most investors do not continue investing beyond their first month
- Retention drops significantly after the initial investment
- Investors who return tend to do so within the first few months

## Limitations

- Analysis is based on event logs only
- No investment amount or loan performance data available
- Calendar month approximation used for duration calculation